In [1]:
import tensorflow as tf
from tensorflow import keras

2023-06-07 16:15:03.486337: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


# Use Nadam Optimization and Early Stopping

In [43]:
data = keras.datasets.cifar10.load_data()

X_train, y_train = data[0]
X_test, y_test = data[1]

X_val = X_train[40000:]
X_train = X_train[:40000]

y_val = y_train[40000:]
y_train = y_train[:40000]

In [52]:
import os
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32,32,3]),
])


for i in range(20):
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

run_index = 1
log_dir = os.path.join(os.curdir, "logs", "run_" + str(run_index))
tensorboard = keras.callbacks.TensorBoard(log_dir)
early_stopping = keras.callbacks.EarlyStopping(patience=10)
rates = [1e-5, 3e-5, 5e-5, 1e-4, 3e-4, 5e-4, 1e-3, 3e-3, 5e-3]
losses = []

untrained_model = keras.models.clone_model(model)

for rate in rates:
    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=rate, beta_1=0.9, beta_2=0.999), 
                loss="sparse_categorical_crossentropy",
                metrics=["accuracy"],)

    model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard, early_stopping])
    losses.append(model.evaluate(X_val, y_val))
    run_index += 1
    log_dir = os.path.join(os.curdir, "logs", "run_" + str(run_index))
    tensorboard = keras.callbacks.TensorBoard(log_dir)
    model = keras.models.clone_model(untrained_model)


Epoch 1/10
1250/1250 [==============================] - 12s 5ms/step - loss: 11.0068 - accuracy: 0.1193 - val_loss: 3.6182 - val_accuracy: 0.1367
Epoch 2/10
1250/1250 [==============================] - 6s 5ms/step - loss: 2.9597 - accuracy: 0.1478 - val_loss: 2.4955 - val_accuracy: 0.1696
Epoch 3/10
1250/1250 [==============================] - 6s 5ms/step - loss: 2.3614 - accuracy: 0.1816 - val_loss: 2.2501 - val_accuracy: 0.1931
Epoch 4/10
1250/1250 [==============================] - 6s 5ms/step - loss: 2.1823 - accuracy: 0.2123 - val_loss: 2.1385 - val_accuracy: 0.2283
Epoch 5/10
1250/1250 [==============================] - 6s 5ms/step - loss: 2.0871 - accuracy: 0.2391 - val_loss: 2.0501 - val_accuracy: 0.2485
Epoch 6/10
1250/1250 [==============================] - 6s 5ms/step - loss: 2.0157 - accuracy: 0.2633 - val_loss: 1.9966 - val_accuracy: 0.2737
Epoch 7/10
1250/1250 [==============================] - 7s 5ms/step - loss: 1.9588 - accuracy: 0.2835 - val_loss: 1.9671 - val_accurac

KeyboardInterrupt: 

In [62]:
import numpy as np
def find_optimal_lr(model: keras.Model, X_train, y_train, X_val, y_val, epochs=10, orders=5):
    bases = [1,3,5]
    rates = []
    for order in range(orders, 0, -1):
        for base in bases:
            rates.append(base/10**order)
    print(rates)
    untrained_model = keras.models.clone_model(model)
    accuracies = []
    for rate in rates:
        model.compile(optimizer=keras.optimizers.Nadam(learning_rate=rate), 
                      loss="sparse_categorical_crossentropy", 
                      metrics=["accuracy"])
        model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
        accuracy = model.evaluate(X_val, y_val)[1]
        accuracies.append(accuracy)
        model = keras.models.clone_model(untrained_model)
    
    return rates[np.argmax(accuracies)]
    
    
    



In [60]:
from sklearn.metrics import accuracy_score
import numpy as np

best_index = np.argmax(np.array(losses)[:, 1])
best_rate = rates[best_index]

model = keras.models.clone_model(untrained_model)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(learning_rate=best_rate),
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), callbacks=[early_stopping])

[0.31940001 0.36359999 0.40099999 0.40970001 0.39399999 0.37830001
 0.3073     0.0952    ]
0.0001
3
Epoch 1/300
1250/1250 [==============================] - 12s 5ms/step - loss: 3.8212 - accuracy: 0.1801 - val_loss: 2.0699 - val_accuracy: 0.2497
Epoch 2/300
1250/1250 [==============================] - 6s 5ms/step - loss: 2.0058 - accuracy: 0.2637 - val_loss: 2.0026 - val_accuracy: 0.2820
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.9133 - accuracy: 0.3049 - val_loss: 1.9354 - val_accuracy: 0.3004
Epoch 4/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.8504 - accuracy: 0.3272 - val_loss: 1.8308 - val_accuracy: 0.3394
Epoch 5/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.8009 - accuracy: 0.3439 - val_loss: 1.7849 - val_accuracy: 0.3566
Epoch 6/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.7562 - accuracy: 0.3619 - val_loss: 1.7752 - val_accuracy: 0.3586
Epoch 7/300
1250/1250 [======

In [61]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 1.5206 - accuracy: 0.4735


[1.5205923318862915, 0.47350001335144043]

# Add Batch Normalization

In [63]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32,32,3]),
])

for i in range(20):
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

early_stopping = keras.callbacks.EarlyStopping(patience=10)


find_optimal_lr(model, X_train, y_train, X_val, y_val)

[1e-05, 3e-05, 5e-05, 0.0001, 0.0003, 0.0005, 0.001, 0.003, 0.005, 0.01, 0.03, 0.05, 0.1, 0.3, 0.5]
Epoch 1/10
1250/1250 [==============================] - 25s 11ms/step - loss: 2.6528 - accuracy: 0.1438 - val_loss: 2.3980 - val_accuracy: 0.1820
Epoch 2/10
1250/1250 [==============================] - 14s 11ms/step - loss: 2.3124 - accuracy: 0.1931 - val_loss: 2.2072 - val_accuracy: 0.2236
Epoch 3/10
1250/1250 [==============================] - 14s 11ms/step - loss: 2.1703 - accuracy: 0.2253 - val_loss: 2.1026 - val_accuracy: 0.2492
Epoch 4/10
1250/1250 [==============================] - 14s 11ms/step - loss: 2.0769 - accuracy: 0.2555 - val_loss: 2.0262 - val_accuracy: 0.2701
Epoch 5/10
1250/1250 [==============================] - 14s 11ms/step - loss: 2.0114 - accuracy: 0.2752 - val_loss: 1.9667 - val_accuracy: 0.2989
Epoch 6/10
1250/1250 [==============================] - 14s 11ms/step - loss: 1.9642 - accuracy: 0.2939 - val_loss: 1.9263 - val_accuracy: 0.3120
Epoch 7/10
1250/1250 [==

0.001

In [64]:

model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), callbacks=[early_stopping])
accuracy_score(y_test, np.argmax(model.predict(X_test), axis=1))

Epoch 1/300
1250/1250 [==============================] - 27s 11ms/step - loss: 1.8846 - accuracy: 0.3239 - val_loss: 1.7553 - val_accuracy: 0.3762
Epoch 2/300
1250/1250 [==============================] - 14s 11ms/step - loss: 1.7267 - accuracy: 0.3835 - val_loss: 1.6815 - val_accuracy: 0.4064
Epoch 3/300
1250/1250 [==============================] - 14s 11ms/step - loss: 1.6576 - accuracy: 0.4109 - val_loss: 1.5881 - val_accuracy: 0.4300
Epoch 4/300
1250/1250 [==============================] - 14s 11ms/step - loss: 1.6059 - accuracy: 0.4282 - val_loss: 1.5361 - val_accuracy: 0.4576
Epoch 5/300
1250/1250 [==============================] - 14s 11ms/step - loss: 1.5612 - accuracy: 0.4467 - val_loss: 1.4918 - val_accuracy: 0.4728
Epoch 6/300
1250/1250 [==============================] - 13s 11ms/step - loss: 1.5186 - accuracy: 0.4610 - val_loss: 1.5042 - val_accuracy: 0.4619
Epoch 7/300
1250/1250 [==============================] - 14s 11ms/step - loss: 1.4877 - accuracy: 0.4757 - val_loss: 1

0.5256

# Self-normalizing Network

In [65]:
from sklearn.preprocessing import StandardScaler
import time

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_val = scaler.fit_transform(X_val.reshape(X_val.shape[0], -1))
X_test = scaler.fit_transform(X_test.reshape(X_test.shape[0], -1))

model = keras.models.Sequential([])

for i in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

early_stopping = keras.callbacks.EarlyStopping(patience=10)

model.compile(optimizer=keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999), 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
st = time.monotonic()
model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), callbacks=[early_stopping])
et = time.monotonic()

print(f"Training took {et-st} seconds")

Epoch 1/300
1250/1250 [==============================] - 12s 5ms/step - loss: 1.9479 - accuracy: 0.2890 - val_loss: 1.8748 - val_accuracy: 0.3192
Epoch 2/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.7511 - accuracy: 0.3696 - val_loss: 1.7307 - val_accuracy: 0.3543
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.6669 - accuracy: 0.4064 - val_loss: 1.6934 - val_accuracy: 0.4029
Epoch 4/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.6123 - accuracy: 0.4267 - val_loss: 1.6808 - val_accuracy: 0.4144
Epoch 5/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.5566 - accuracy: 0.4474 - val_loss: 1.6141 - val_accuracy: 0.4330
Epoch 6/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.5155 - accuracy: 0.4636 - val_loss: 1.6207 - val_accuracy: 0.4234
Epoch 7/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.4737 - accuracy: 0.4788 - val_loss: 1.5596 - val_a

In [66]:

accuracy_score(y_test, np.argmax(model.predict(X_test), axis=1))

313/313 [==============================] - 0s 1ms/step


0.4665

# Regularizing with Alpha Dropout

In [68]:
import time
import os

model = keras.models.Sequential([])

for i in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

early_stopping = keras.callbacks.EarlyStopping(patience=10)
run_index = 1
tensorboard = keras.callbacks.TensorBoard(log_dir=os.path.join(os.curdir, "logs", "run_" + str(run_index)))

model.compile(optimizer=keras.optimizers.Nadam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999), 
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

st = time.monotonic()
model.fit(X_train, y_train, epochs=300, validation_data=(X_val, y_val), callbacks=[early_stopping, tensorboard])
et = time.monotonic()

print(f"Training took {et-st} seconds")

Epoch 1/300
1250/1250 [==============================] - 12s 5ms/step - loss: 1.9167 - accuracy: 0.3183 - val_loss: 1.8205 - val_accuracy: 0.3753
Epoch 2/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.6931 - accuracy: 0.3970 - val_loss: 1.7574 - val_accuracy: 0.4057
Epoch 3/300
1250/1250 [==============================] - 6s 5ms/step - loss: 1.6076 - accuracy: 0.4331 - val_loss: 1.6905 - val_accuracy: 0.4194
Epoch 4/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.5385 - accuracy: 0.4568 - val_loss: 1.6204 - val_accuracy: 0.4493
Epoch 5/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.4813 - accuracy: 0.4794 - val_loss: 1.6080 - val_accuracy: 0.4580
Epoch 6/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.4383 - accuracy: 0.4987 - val_loss: 1.6389 - val_accuracy: 0.4698
Epoch 7/300
1250/1250 [==============================] - 7s 5ms/step - loss: 1.3944 - accuracy: 0.5151 - val_loss: 1.6661 - val_a

In [69]:
accuracy_score(y_test, np.argmax(model.predict(X_test), axis=1))

313/313 [==============================] - 0s 1ms/step


0.5019

In [72]:
y_probas = np.stack([model(X_test, training=True) for sample in range(500)])
y_proba = y_probas.mean(axis=0)

In [73]:
accuracy_score(y_test, np.argmax(y_proba, axis=1))

0.5022

In [75]:
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        keras.backend.set_value(self.model.optimizer.learning_rate, rate)

In [80]:
import math

epochs = 15

model = keras.models.Sequential([])
for i in range(20):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
model.add(keras.layers.AlphaDropout(rate=0.2))
model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_normal"))

onecycle = OneCycleScheduler(math.ceil(len(X_train) / 32) * epochs, 0.05)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-2),
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), callbacks=[onecycle, early_stopping])

Epoch 1/15
1250/1250 [==============================] - 7s 4ms/step - loss: 1.9646 - accuracy: 0.3063 - val_loss: 1.7703 - val_accuracy: 0.3950
Epoch 2/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6820 - accuracy: 0.4010 - val_loss: 1.7144 - val_accuracy: 0.4164
Epoch 3/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6092 - accuracy: 0.4292 - val_loss: 1.7010 - val_accuracy: 0.4210
Epoch 4/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.5610 - accuracy: 0.4503 - val_loss: 1.7420 - val_accuracy: 0.4317
Epoch 5/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.5265 - accuracy: 0.4608 - val_loss: 1.7159 - val_accuracy: 0.4207
Epoch 6/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.4997 - accuracy: 0.4740 - val_loss: 1.8140 - val_accuracy: 0.4314
Epoch 7/15
1250/1250 [==============================] - 5s 4ms/step - loss: 1.4784 - accuracy: 0.4836 - val_loss: 1.7156 - val_accuracy:

In [81]:
accuracy_score(y_test, np.argmax(model.predict(X_test), axis=1))

313/313 [==============================] - 0s 1ms/step


0.5312